# Esercizio1 - Pysam

Prendere in input un BAM file che contiene allineamenti ad un'unica reference (cromosoma X) e:

- controllare se sono presenti *paired-end* reads
- determinare il numero e le lunghezze di tutti gli introni supportati dagli allineamenti del BAM file
- determinare la base della reference a copertura massima e produrre un file SAM contenente i soli allineamenti che coprono tale base

Importare il modulo `pysam`

In [4]:
import pysam

In [5]:
from pysam import AlignmentFile

Caricare il file `BAM` in input

In [6]:
pysam.index('./sample.bam')
bamfile = AlignmentFile('./sample.bam', 'rb')

#### Controllare se ci sono dei reads paired-end.

In [7]:
alignment_list = list(bamfile.fetch())

In [8]:
any(alignment.is_paired for alignment in alignment_list)

False

#### Determinare il numero e le lunghezze di tutti gli introni supportati dagli allineamenti

Determinare la lista delle lunghezze degli introni supportati con duplicati.

In [9]:
spliced_cigars = [alignment.cigarstring for alignment in alignment_list if 'N' in alignment.cigarstring]

In [10]:
import re

intron_lengths = [re.findall('(\d+)N', cigar)[0] for cigar in spliced_cigars]

Determinare il set delle lunghezze degli introni supportati.

In [11]:
set(intron_lengths)

{'1514', '1999', '287', '309', '4116', '4226', '57', '598', '980'}

#### Trovare la base a copertura massima e produrre il SAM file contenente tutti gli allineamenti che coprono questa base.

Determinare la lista delle *pileup columns*.

In [19]:
pileup_iter = bamfile.pileup()

In [20]:
pileup_columns = list(pileup_iter)

Estrarre la colonna di *pileup* con il maggior numero di reads.

In [21]:
max_col = max(pileup_col.nsegments for pileup_col in pileup_columns)

In [22]:
max_pileup_col = [pileup_col for pileup_col in pileup_columns if pileup_col.nsegments == max_col][0]

In [23]:
max_pileup_col.nsegments

1469

Settare a 0 la qualità minima della colonna di *pileup* (altrimenti le reads non vengono tenute perché hanno una qualità bassa).

In [24]:
max_pileup_col.set_min_base_quality(0)

Estrarre la lista dei read di *pileup* relativi alla colonna di altezza massima.

In [25]:
max_pileup_col.pileups

 ...]

Estrarre la lista degli allineamenti relativi alla colonna di altezza massima.

In [29]:
pileup_slignments = [pileup_read.alignment for pileup_read in max_pileup_col.pileups]

Produrre gli allineamenti in un SAM file utilizzando la stessa Header Section del BAM file.

In [30]:
outputfile = pysam.AlignmentFile('./prova.sam', "w", template=bamfile)

for read in pileup_slignments:
    outputfile.write(read)
    
outputfile.close()